# Add labels to DB
For every _valid_ experiment, add labels to a separate table.
We use the following labels:
- scaled MS responses for all products
- binary outcome for all products
- major product (A, B, C, or no_product)

We use the methods for scaling that were established in the exploratory data analysis.

## Scaling the LCMS responses
- It seems to make sense to use some kind of robust scaling as we definitely expect outliers in LCMS data.
- We do not require centering, as the data starts at 0 and is not normally distributed.
- We could divide by the interquartile range (IQR) to get a robust scaling.
- We could also use the median absolute deviation (MAD) to get a robust scaling.
- However, both above ideas seem more suitable for symmetric distributions, while we have a highly skewed distribution with a defined minimum at 0.
- Therefore, we could just divide by, say, the 90th percentile to get the equivalent of min-max scaling but robust to outliers.
- Which percentile we choose, depends on the number of outliers we expect. The 90th percentile seems reasonable as we already remove internal standard errors and we can expect less than 10% of compounds to be outliers by virtue of ionizability alone. At the same time, choosing the 90th means we expect all products to form in at least 10% of the reactions.
- To not be dependent on how often a product forms: Use a lower (say 80th) percentile, but use only non-zero values in calculating the percentile.

### An extension to the scaling idea
Assume we have achieved robust scaling. There remains a problem:
- Consider product A systematically ionizes 10 times better than product B.
- Further, consider a reaction where we measure a non-zero response for product A that is at the lower detection limit
- It follows that we have no way to tell whether product B was formed or not, we only know it was not formed 10 times as much as A.

In general, if product A ionized better than product B by a factor of x, we need to see a response for product A exceeding the lower detection limit by a factor of x to conclude that product A is the major product.


In [ ]:
import pathlib
import sys
from typing import Tuple, Union

sys.path.append(str(pathlib.Path().resolve().parents[1]))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.util.db_utils import SynFermDatabaseConnection
from src.definitions import DATA_DIR

In [ ]:
con = SynFermDatabaseConnection()

In [ ]:
# get only valid experiments from DB
data = con.con.execute("SELECT * FROM experiments WHERE exp_nr BETWEEN 4 AND 29 AND (valid NOT LIKE '%ERROR%' OR valid IS NULL)").fetchall()
df = pd.DataFrame(data, columns=[c[1] for c in con.con.execute("PRAGMA table_info(experiments)").fetchall()])
len(df)

In [ ]:
def robust_scale(x: np.ndarray, percentile: int = 75, return_scaling_factor=False) -> Union[np.ndarray, Tuple[np.ndarray, float]]:
    """
    Robust scaling of a numpy array by dividing by the value for a percentile.
    The percentile is calculated from all non-zero values.
    """
    x = x.copy()
    scaling_factor = np.percentile(x[x > 0], percentile)
    x /= scaling_factor
    if return_scaling_factor:
        return x, scaling_factor
    else:
        return x

In [ ]:
# before we apply scaling, we need to verify how sensitive the scaling is to our choice of the percentile.
factors = []
for percentile in range(1, 100):
    scaled_arrs = []
    scaling_factors = []
    for s in "ABCDEFGH":
        arr, factor = robust_scale(df[f'product_{s}_lcms_ratio'].values, percentile=percentile, return_scaling_factor=True)
        scaled_arrs.append(arr)
        scaling_factors.append(factor)
    factors.append(scaling_factors / scaling_factors[0])

In [ ]:
# plot the relative scaling factors for each product (A is always 1) for all percentiles between [1, 99]
plt.figure(figsize=(7.5,5))
plt.plot(factors)
plt.xlabel("Percentile")
plt.ylabel("Scaling factor, relative to A")
plt.axvline(85, color="red", linestyle="--", label="85th")
plt.legend("ABCDEFGH")


### Result of the percentile/scaling factor analysis
This outcome is good. The range of the scaling factors is mostly 2x - 3x. This seems reasonable given that the molecules always share quite a few residues. Also, except product D, all relative factors are quite stable w.r.t. the choice of percentile. In particular, relative A,B,C are stable. As we expect, in the very high percentiles (>95 for most, >90 for E), the scaling factor becomes unstable as it is now dominated by outliers.

**Based on the plot, we choose the 85th percentile**

In [ ]:
# apply scaling to all products and show scaling factors for control
scaling_factors = []
for s in "ABCDEFGH":
        arr, factor = robust_scale(df[f'product_{s}_lcms_ratio'].values, percentile=85, return_scaling_factor=True)
        df[f'scaled_{s}'] = arr
        scaling_factors.append(factor)
print("Scaling factors relative to A:")
for s, i in zip("ABCDEFGH", scaling_factors):
    print(f"{s}: {i/scaling_factors[0]:.3f}")
# save the scaling factors for later use
pd.DataFrame(scaling_factors, columns=["factor"], index=list("ABCDEFGH"))\
    .reset_index(names="product_type")\
    .to_csv(DATA_DIR / "scaling-factors_2023-12-20.csv", index=False)

In [ ]:
# add binary outcome: 1 if product was formed, 0 if not
df["binary_A"] = (df["scaled_A"] > 0).astype(int)
df["binary_B"] = (df["scaled_B"] > 0).astype(int)
df["binary_C"] = (df["scaled_C"] > 0).astype(int)
df["binary_D"] = (df["scaled_D"] > 0).astype(int)
df["binary_E"] = (df["scaled_E"] > 0).astype(int)
df["binary_F"] = (df["scaled_F"] > 0).astype(int)
df["binary_G"] = (df["scaled_G"] > 0).astype(int)
df["binary_H"] = (df["scaled_H"] > 0).astype(float)
df.loc[df["scaled_H"].isna(), "binary_H"] = pd.NA

In [ ]:
df.describe()

In [ ]:
# assign the main product
df["main_product"] = df[["scaled_A", "scaled_B", "scaled_C"]].idxmax(axis=1).str.replace("scaled_", "")
# are there any reactions where neither A,nor B, nor C appear?
df.loc[df[["scaled_A", "scaled_B", "scaled_C"]].sum(axis=1) == 0, "main_product"] = "no_product"
df

In [ ]:
# write to 'labels' table of DB
with con.con:
    con.con.executemany('INSERT INTO labels (experiment_id, scaled_A, scaled_B, scaled_C, scaled_D, scaled_E, scaled_F, scaled_G, scaled_H, binary_A, binary_B, binary_C, binary_D, binary_E, binary_F, binary_G, binary_H, "major_A-C") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);',
                        df[["id", "scaled_A", "scaled_B", "scaled_C", "scaled_D", "scaled_E", "scaled_F", "scaled_G", "scaled_H", "binary_A", "binary_B", "binary_C", "binary_D", "binary_E", "binary_F", "binary_G", "binary_H", "main_product"]].values.tolist()
                        )